# PPT02

In [1]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/Owner/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
!pip install --upgrade gensim
!pip install -U spacy
!pip install -U pip setuptools wheel
!pip install pyldavis

In [3]:
!python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 13.6 MB 327 kB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
!pip install pyLDAvis==3.2.1

In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.models
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [6]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [8]:
PPT01 = pd.read_csv('PPT01.csv')
PPT01 = PPT01.loc[PPT01['Person'] != 'Interviewer']

In [9]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(PPT01['Question/Answer']))

print(data_words[:1])

[['to', 'pay', 'monthly', 'utility', 'both', 'electricity', 'and', 'water']]


In [10]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['to', 'pay', 'monthly', 'utility', 'both', 'electricity', 'and', 'water']


In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['pay', 'monthly', 'utility', 'electricity', 'water']]


In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [14]:
id2word[0]

'electricity'

In [15]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('electricity', 1),
  ('monthly', 1),
  ('pay', 1),
  ('utility', 1),
  ('water', 1)]]

In [16]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [17]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.054*"user" + 0.054*"really" + 0.036*"question" + 0.036*"process" + '
  '0.036*"agent" + 0.036*"function" + 0.036*"exactly" + 0.019*"take" + '
  '0.019*"pay" + 0.019*"time"'),
 (1,
  '0.061*"mobile" + 0.041*"try" + 0.041*"also" + 0.041*"website" + '
  '0.021*"good" + 0.021*"correctly" + 0.021*"help" + 0.021*"verify" + '
  '0.021*"call" + 0.021*"customer"'),
 (2,
  '0.061*"go" + 0.031*"take" + 0.031*"want" + 0.031*"people" + 0.031*"suggest" '
  '+ 0.031*"apply" + 0.031*"reach" + 0.031*"office" + 0.031*"service" + '
  '0.031*"day"'),
 (3,
  '0.098*"website" + 0.066*"issue" + 0.034*"service" + 0.034*"happen" + '
  '0.034*"know" + 0.034*"day" + 0.034*"desirable" + 0.034*"end" + '
  '0.034*"often" + 0.034*"unfortunately"'),
 (4,
  '0.005*"agent" + 0.005*"exactly" + 0.005*"available" + 0.005*"amount" + '
  '0.005*"along" + 0.005*"bubble" + 0.005*"actually" + 0.005*"science" + '
  '0.005*"suggest" + 0.005*"clearly"'),
 (5,
  '0.006*"customer" + 0.005*"form" + 0.005*"people" + 0.005*"

In [18]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.815941920366146

Coherence Score:  0.6297991418087121


In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.124587  0.248463       1        1  14.190769
12     0.075203 -0.258926       2        1  13.225017
17    -0.236243 -0.165005       3        1  13.005321
0     -0.179134  0.208092       4        1   9.403044
11     0.198812 -0.161369       5        1   9.206415
10     0.248654 -0.058286       6        1   8.701044
1     -0.028214  0.250986       7        1   8.059655
19    -0.118919 -0.206640       8        1   6.042852
2     -0.215199  0.041988       9        1   4.884280
3      0.192089  0.111816      10        1   4.267381
9      0.013983 -0.159307      11        1   4.078253
6      0.103504  0.010989      12        1   1.720199
8     -0.095258 -0.011358      13        1   1.425383
18    -0.056136  0.048563      14        1   0.893110
5     -0.003993  0.017618      15        1   0.151686
16    -0.004938  0.017089      16        1   0.151576
15    -0.004931  0.015721      17        1   0.151513
4     -0.004622  0.016522      18        1   0.147501
13    -0.004622  0.016522      19        1   0.147501
14    -0.004622  0.016522      20        1   0.147501, topic_info=        Term       Freq      Total Category  logprob  loglift
21      form  16.000000  16.000000  Default  30.0000  30.0000
2        pay  14.000000  14.000000  Default  29.0000  29.0000
170  website   7.000000   7.000000  Default  28.0000  28.0000
7       time   5.000000   5.000000  Default  27.0000  27.0000
105     bill   6.000000   6.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
25   support   0.003245   4.343714  Topic20  -5.3706  -0.6803
26       use   0.003245   5.879841  Topic20  -5.3706  -0.9831
27    around   0.003245   1.290172  Topic20  -5.3706   0.5337
28      care   0.003245   4.568893  Topic20  -5.3706  -0.7308
29       day   0.003245   2.625888  Topic20  -5.3706  -0.1770

[969 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
196       8  0.824319    access
112       3  0.747056   account
202       4  0.782824    actual
141       1  0.736531  actually
104       3  0.914718       add
...     ...       ...       ...
80        2  0.470025      work
80        4  0.470025      work
42        2  0.467280     wrong
42        5  0.467280     wrong
111       3  0.745349       yet

[343 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 13, 18, 1, 12, 11, 2, 20, 3, 4, 10, 7, 9, 19, 6, 17, 16, 5, 14, 15])

In [21]:
pyLDAvis.save_html(vis, 'lda.html')

# PPT02

In [22]:
PPT02 = pd.read_csv('PPT02.csv')
PPT02 = PPT02.loc[PPT02['Person'] != 'Interviewer']

In [23]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(PPT02['Question/Answer']))

print(data_words[:1])

[['to', 'pay', 'my', 'bills', 'and', 'to', 'get', 'an', 'estimate', 'of', 'my', 'upcoming', 'bills', 'so', 'that', 'can', 'put', 'in', 'the', 'inaudible', 'meter', 'readings', 'and', 'get', 'an', 'estimate', 'of', 'my', 'monthly', 'bills', 'that', 'and', 'to', 'check', 'whether', 'there', 'are', 'any', 'interruptions', 'in', 'the', 'services']]


In [24]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['to', 'pay', 'my', 'bills', 'and', 'to', 'get', 'an', 'estimate', 'of', 'my', 'upcoming', 'bills', 'so', 'that', 'can', 'put', 'in', 'the', 'inaudible', 'meter', 'readings', 'and', 'get', 'an', 'estimate', 'of', 'my', 'monthly', 'bills', 'that', 'and', 'to', 'check', 'whether', 'there', 'are', 'any', 'interruptions', 'in', 'the', 'services']


In [25]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [26]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['pay', 'bill', 'get', 'estimate', 'upcoming', 'bill', 'put', 'inaudible', 'meter', 'reading', 'get', 'estimate', 'monthly', 'bill', 'check', 'interruption', 'service']]


In [27]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 3), (1, 1), (2, 2), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]]


In [28]:
id2word[0]

'bill'

In [29]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('bill', 3),
  ('check', 1),
  ('estimate', 2),
  ('get', 2),
  ('inaudible', 1),
  ('interruption', 1),
  ('meter', 1),
  ('monthly', 1),
  ('pay', 1),
  ('put', 1),
  ('reading', 1),
  ('service', 1),
  ('upcoming', 1)]]

In [30]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [31]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.082*"definitely" + 0.082*"bill" + 0.042*"respect" + 0.042*"provider" + '
  '0.042*"think" + 0.042*"facility" + 0.042*"choose" + 0.042*"kinda" + '
  '0.042*"payment" + 0.042*"point"'),
 (1,
  '0.052*"customer" + 0.039*"account" + 0.026*"number" + 0.026*"quite" + '
  '0.026*"login" + 0.026*"really" + 0.026*"advise" + 0.026*"know" + '
  '0.026*"say" + 0.026*"call"'),
 (2,
  '0.006*"least" + 0.006*"phonecall" + 0.006*"acknowledge" + 0.006*"come" + '
  '0.006*"expect" + 0.006*"good" + 0.006*"government" + 0.006*"happen" + '
  '0.006*"inform" + 0.006*"installation"'),
 (3,
  '0.006*"least" + 0.006*"phonecall" + 0.006*"acknowledge" + 0.006*"come" + '
  '0.006*"expect" + 0.006*"good" + 0.006*"government" + 0.006*"happen" + '
  '0.006*"inform" + 0.006*"installation"'),
 (4,
  '0.006*"least" + 0.006*"phonecall" + 0.006*"acknowledge" + 0.006*"come" + '
  '0.006*"expect" + 0.006*"good" + 0.006*"government" + 0.006*"happen" + '
  '0.006*"inform" + 0.006*"installation"'),
 (5,
  '0.085*"sa

In [32]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.7116120012684

Coherence Score:  0.5775797967788939


In [33]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13     0.157419  0.235315       1        1  18.681467
1      0.001472 -0.280407       2        1  15.964321
17    -0.257469 -0.116315       3        1  13.255681
9     -0.199799  0.214787       4        1  12.492297
11     0.183052 -0.191317       5        1   9.598212
8      0.217582 -0.077557       6        1   7.003990
10    -0.064063  0.193646       7        1   4.161691
12    -0.137641 -0.154031       8        1   3.930408
0     -0.180740  0.042301       9        1   3.699307
5      0.168956  0.089213      10        1   3.362359
16     0.146859  0.004250      11        1   2.779605
15     0.053496  0.141544      12        1   2.444140
19    -0.082490 -0.014521      13        1   1.393228
6      0.030756 -0.021363      14        1   0.490294
18    -0.005804 -0.011077      15        1   0.126713
7     -0.006661 -0.010987      16        1   0.126648
4     -0.006232 -0.010871      17        1   0.122410
3     -0.006232 -0.010871      18        1   0.122410
14    -0.006232 -0.010871      19        1   0.122410
2     -0.006232 -0.010871      20        1   0.122410, topic_info=           Term       Freq      Total Category  logprob  loglift
29         form  11.000000  11.000000  Default  30.0000  30.0000
11      service  12.000000  12.000000  Default  29.0000  29.0000
14        think   6.000000   6.000000  Default  28.0000  28.0000
28         fill   7.000000   7.000000  Default  27.0000  27.0000
0          bill   4.000000   4.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
25     download   0.002922   3.167918  Topic20  -5.1985  -0.2831
26  electricity   0.002922   1.377863  Topic20  -5.1985   0.5494
27        email   0.002922   3.133434  Topic20  -5.1985  -0.2722
28         fill   0.002922   7.297605  Topic20  -5.1985  -1.1176
29         form   0.002922  11.769152  Topic20  -5.1985  -1.5956

[932 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
141       1  0.725296         able
50        2  0.474871      account
50        5  0.316581      account
50        6  0.158290      account
116       3  0.744989  acknowledge
...     ...       ...          ...
18       12  0.301356      website
133       3  0.745369         week
45        1  0.633378         well
45        2  0.316689         well
159       1  0.725221         year

[312 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 2, 18, 10, 12, 9, 11, 13, 1, 6, 17, 16, 20, 7, 19, 8, 5, 4, 15, 3])

In [34]:
pyLDAvis.save_html(vis, 'lda.html')

# All

In [34]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/Owner/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
!pip install --upgrade gensim
!pip install -U spacy
!pip install -U pip setuptools wheel
!pip install pyldavis

In [36]:
!python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 13.6 MB 246 kB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [37]:
!pip install pyLDAvis==3.2.1

In [38]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.models
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [39]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [35]:
df = pd.read_csv('All.csv')
df = df.loc[df['Person'] != 'Interviewer']

In [36]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df['Question/Answer']))

print(data_words[:1])

[['to', 'pay', 'monthly', 'utility', 'both', 'electricity', 'and', 'water']]


In [37]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['to', 'pay', 'monthly', 'utility', 'both', 'electricity', 'and', 'water']


In [38]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [39]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['pay', 'monthly', 'utility', 'electricity', 'water']]


In [40]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [41]:
id2word[0]

'electricity'

In [42]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('electricity', 1),
  ('monthly', 1),
  ('pay', 1),
  ('utility', 1),
  ('water', 1)]]

In [43]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [44]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.051*"really" + 0.034*"long" + 0.034*"think" + 0.034*"mean" + '
  '0.034*"function" + 0.034*"user" + 0.034*"agent" + 0.018*"take" + '
  '0.018*"month" + 0.018*"install"'),
 (1,
  '0.064*"bill" + 0.064*"definitely" + 0.033*"point" + 0.033*"kinda" + '
  '0.033*"mean" + 0.033*"really" + 0.033*"choose" + 0.033*"facility" + '
  '0.033*"pay" + 0.033*"provider"'),
 (2,
  '0.061*"think" + 0.049*"month" + 0.037*"account" + 0.037*"login" + '
  '0.025*"take" + 0.025*"roughly" + 0.025*"pay" + 0.025*"utility" + '
  '0.025*"get" + 0.025*"actually"'),
 (3,
  '0.049*"find" + 0.049*"initially" + 0.048*"bill" + 0.048*"fill" + '
  '0.048*"form" + 0.048*"pay" + 0.003*"login" + 0.003*"account" + 0.003*"open" '
  '+ 0.003*"knowledge"'),
 (4,
  '0.062*"inaudible" + 0.003*"trigger" + 0.003*"estimate" + 0.003*"viable" + '
  '0.003*"solution" + 0.003*"physical" + 0.003*"option" + 0.003*"check" + '
  '0.003*"interruption" + 0.003*"smooth"'),
 (5,
  '0.161*"form" + 0.052*"apply" + 0.030*"service" + 0.030

In [45]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.109806486355007

Coherence Score:  0.477350908821094


In [46]:
#import pyLDAvis.gensim AS gensimvis

In [47]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.087285  0.216163       1        1  15.852692
10     0.039682 -0.287748       2        1  14.202920
7      0.187309  0.187935       3        1  11.284527
6     -0.200757 -0.185140       4        1  10.786512
16    -0.220321  0.158904       5        1   8.282088
9      0.043184  0.262238       6        1   7.699648
2      0.245870 -0.083468       7        1   7.173050
0     -0.233023 -0.090264       8        1   4.873438
11     0.247885  0.054807       9        1   4.265791
8     -0.166084  0.089681      10        1   3.936863
12    -0.057638 -0.160581      11        1   2.888398
13     0.102896 -0.138672      12        1   2.337227
5     -0.123545  0.017736      13        1   2.263718
1      0.111964 -0.066454      14        1   1.838816
19     0.068922  0.040936      15        1   0.955896
3      0.007690  0.033362      16        1   0.744427
4      0.012512 -0.017685      17        1   0.208554
17     0.006517 -0.010605      18        1   0.139057
14     0.007110 -0.010572      19        1   0.133190
15     0.007110 -0.010572      20        1   0.133190, topic_info=          Term       Freq      Total Category  logprob  loglift
21        form  27.000000  27.000000  Default  30.0000  30.0000
2          pay  17.000000  17.000000  Default  29.0000  29.0000
105       bill  10.000000  10.000000  Default  28.0000  28.0000
216  inaudible   6.000000   6.000000  Default  27.0000  27.0000
18        fill  13.000000  13.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
25     support   0.003802   6.399114  Topic20  -5.7589  -0.8071
26         use   0.003802   6.799955  Topic20  -5.7589  -0.8679
27      around   0.003802   1.426002  Topic20  -5.7589   0.6942
28        care   0.003802   6.142554  Topic20  -5.7589  -0.7662
29         day   0.003802   5.619477  Topic20  -5.7589  -0.6772

[1007 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
290       2  0.689967     able
196       2  0.689928   access
112       3  0.282841  account
112       6  0.282841  account
112       7  0.282841  account
...     ...       ...      ...
80        8  0.455087     work
42        1  0.431275    wrong
42        4  0.431275    wrong
300       2  0.689963     year
111       5  0.716824      yet

[590 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 11, 8, 7, 17, 10, 3, 1, 12, 9, 13, 14, 6, 2, 20, 4, 5, 18, 15, 16])

In [49]:
pyLDAvis.save_html(vis, 'lda.html')